In [ ]:
import numpy as np
np.set_printoptions(suppress=True)
from matplotlib import pyplot as plt
import seaborn
import time
from scipy.io import loadmat
from numpy import fft

## Class Definitions for ANN

In [ ]:
class NetworkLayer:
    def __init__(self, num_units, size_input=None, a_fcn ='sigmoid'):
        self.num_units = num_units
        self.size_input = size_input
        self.layer_weights = None
        self.a_fcn = a_fcn
        self.h_a = None  
        
    def initialize_weights(self):
        self.layer_weights = np.random.randn(self.num_units, self.size_input+1)
        
    def activate(self, net, a_fcn):
        if a_fcn == 'sigmoid':
            return 1 / (1 + np.exp(-net))
        elif a_fcn == 'relu':
            return np.maximum(0.001*net,net)
        elif a_fcn == 'tanh':
            return (np.exp(net)-np.exp(-net))/(np.exp(net)+np.exp(-net))
        elif a_fcn == 'none':
            return net
        
    def derivate(self, h_a, a_fcn):
        if a_fcn == 'sigmoid':
            return h_a * (1 - h_a)
        elif a_fcn == 'relu':
            return np.where(h_a < 0.0, 0.001*h_a, 1.0)
        elif a_fcn == 'tanh':
            return 1-h_a**2
        elif a_fcn == 'none':
            return 1 # NOT SURE
     
    def backprop_layer_error(self, delta, h_a):
        derivative = self.derivate(h_a, self.a_fcn)
        #print('derivative', derivative.shape)
        #print('delta',delta.shape)
        #print('weights',self.layer_weights.shape)
        return np.matmul(delta,self.layer_weights)[:,1:] * derivative # trimmed to get rid of the bias term 
    
    def __call__(self, X):
        num_samples = X.shape[0]
        layer_input = np.hstack([np.ones((num_samples,1)), X]) #add 1s for the bias term
        net = np.matmul(layer_input, self.layer_weights.T)
        
        h_a = self.activate(net, self.a_fcn)
        self.h_a = h_a #storing every hidden layer output
        return h_a

In [ ]:
class ArtificialNeuralNetwork:
    def __init__(self, network_layers, learning_rate):
        self.learning_rate = learning_rate 
        layer_input_size = network_layers[0].num_units
        network_layers[0].initialize_weights()
         
        for layer in network_layers[1:]:
            layer.size_input = layer_input_size
            layer_input_size = layer.num_units
            layer.initialize_weights()
            
        self.network_layers = network_layers
    
    def forwardpropagate(self, X):
        layer_output = self.network_layers[0](X)
        for layer in self.network_layers[1:]:
            layer_output = layer(layer_output)
        return layer_output
    
    def bias(self,vec):
        return np.hstack([np.ones((vec.shape[0], 1)), vec])
    
    def backpropagation(self, X, regression_output, true):
        n_layers = len(self.network_layers)
        delta = regression_output - true
        n_a = regression_output
        dWs = {}
        
        for i in range(-1,-len(self.network_layers),-1):
            n_a = self.network_layers[i-1].h_a
            
            dWs[i] = np.matmul(delta.T,self.bias(n_a))
            delta = self.network_layers[i].backprop_layer_error(delta,n_a)
        
        dWs[-n_layers] = np.matmul(delta.T,self.bias(X))
        
        for i_layer, dW in dWs.items():
            self.network_layers[i_layer].layer_weights -= self.learning_rate * dW
    

## Loading Data

In [ ]:
training_data = np.load('./train_data_kspace.npz')
k_train = training_data['train']
#y_train = y_train[:,np.newaxis]
test_data = np.load('./test_data_kspace.npz')
k_test = test_data['test']
#y_test = y_test[:,np.newaxis]

In [ ]:
k_train_concat = ((k_train[1,:65536]+1j*k_train[1,65536:131072]))
k_test_concat = ((k_test[3,:65536]+1j*k_test[3,65536:131072]))

In [ ]:
sample_k = np.reshape(k_test_concat,(256,256))
sample_im = ifft2c(sample_k)
plt.imshow(np.absolute(sample_im), cmap= 'gray')
plt.axis('off')

In [ ]:
k_train_real = k_train[:,:65536]
k_train_imag = k_train[:,65536:131072]
k_test_real = k_test[:,:65536]
k_test_imag = k_test[:,65536:131072]

In [ ]:
x = loadmat('ixi-t1_final.mat')['imdata']
imdata = np.array(np.reshape(x,(581,256*256)))
#imdata.shape
im_train_true = imdata[:500,:]
im_test_true = imdata[500:581,:]

plt.imshow(np.absolute(x[1,:,:]), cmap= 'gray')
plt.axis('off')

In [ ]:
def fft2c(im):
    d = fft.fftshift(fft.fft2(fft.ifftshift(im)))
    return np.real(d), np.imag(d)

def ifft2c(d):
    im = fft.fftshift(fft.ifft2(fft.ifftshift(d)))
    return im

In [ ]:
k_train_true_real , k_train_true_imag = fft2c(im_train_true)
#k_train_true = np.concatenate((k_train_true_real, k_train_true_imag), axis = 1)

k_test_true_real, k_test_true_imag = fft2c(im_test_true)
#k_test_true = np.concatenate((k_test_true_real, k_test_true_imag), axis = 1)

### Cross Validation Split

In [ ]:
# Cross validation function that seperates the training set into k folds and returns them
def create_folds(k_train, k_true, num_folds):
    permutation = np.random.permutation(k_train.shape[0])
    k_train = k_train[permutation]
    im_true = k_true[permutation]
    
    fold_size = int(k_train.shape[0]/num_folds)
    k_train_cv = np.zeros((num_folds,fold_size,k_train.shape[1]))
    #print(X_cv.shape)
    k_true_cv = np.zeros((num_folds,fold_size,k_true.shape[1]))
    #print(y_cv.shape)
    
    #i_sample = 0
    for i_fold in range(num_folds):
        for i_sample in range(fold_size):
            sample_index = i_fold*fold_size + i_sample
            #print(sample_index)
            k_train_cv[i_fold,i_sample,:] = k_train[sample_index,:]
            k_true_cv[i_fold,i_sample,:] = k_true[sample_index,:]
        
        print('Average of fold %d is %f' %(i_fold, np.average(k_train_cv[i_fold,:,:])))
        print('Std of fold %d is %f' %(i_fold, np.std(k_train_cv[i_fold,:,:])))
    return k_train_cv,k_true_cv

k_train_true_real_cv, k_train_true_imag_cv = create_folds(k_train_true_real,k_train_true_imag,10)
k_test_true_real_cv, k_test_true_imag_cv = create_folds(k_test_true_real,k_test_true_imag,10)
k_train_real_cv, k_train_imag_cv = create_folds(k_train_real, k_train_imag,10)
k_test_real_cv, k_test_imag_cv = create_folds(k_test_real, k_test_imag,10)

In [ ]:
N = k_train_real.shape[1]
model_real = ArtificialNeuralNetwork([
    NetworkLayer(8, a_fcn='sigmoid', size_input=N),
    NetworkLayer(16, a_fcn='sigmoid'),
    NetworkLayer(N, a_fcn='none'),
],learning_rate = 0.001)

N = k_train_imag.shape[1]
model_imag = ArtificialNeuralNetwork([
    NetworkLayer(8, a_fcn='sigmoid', size_input=N),
    NetworkLayer(16, a_fcn='sigmoid'),
    NetworkLayer(N, a_fcn='none'),
],learning_rate = 0.001)


### Train with Cross Validation

In [ ]:
num_folds = 10
for e in range(10):
    print('============EPOCH %d ============' %(e))
    for i_fold in range(num_folds):
        print('**********FOLD %d **********' %(i_fold))
        k_cv_test = k_train_real_cv[i_fold,:,:]
        k_cv_test_true = k_train_true_real_cv[i_fold,:]
        k_cv_train = np.concatenate((k_train_real_cv[:i_fold,:,:],k_train_real_cv[i_fold+1:num_folds,:,:]))
        k_cv_train_true = np.concatenate((k_train_true_real_cv[:i_fold,:],k_train_true_real_cv[i_fold+1:num_folds,:]))
        
        reg_out_real = model_real.forwardpropagate(k_cv_train)
        model_real.backpropagation(k_cv_train, reg_out_real, k_cv_train_true)
        
        print('Training with fold %d' %(i_fold))
        print('Training Error (SSE) %f' %(np.sum((k_cv_train_true-reg_out_real)**2)/k_cv_train_true.shape[0]))
        print('Testing with fold %d' %(i_fold))
        
        reg_out_real = model_real.forwardpropagate(k_cv_test)
        model_real.backpropagation(k_cv_fest, reg_out_real, k_cv_test_true)
        print('Test Error (SSE) %f' %(np.sum((k_cv_test-reg_out_real)**2)/k_cv_test.shape[0]))
        
        k_cv_test = k_train_imag_cv[i_fold,:,:]
        k_cv_test_true = k_train_true_imag_cv[i_fold,:]
        k_cv_train = np.concatenate((k_train_imag_cv[:i_fold,:,:],k_train_imag_cv[i_fold+1:num_folds,:,:]))
        k_cv_train_true = np.concatenate((k_train_true_imag_cv[:i_fold,:],k_train_true_imag_cv[i_fold+1:num_folds,:]))
        
        reg_out_imag = model_imag.forwardpropagate(k_cv_train)
        model_imag.backpropagation(k_cv_train, reg_out_imag, k_cv_train_true)
        print('Training with fold %d' %(i_fold))
        print('Training Error (SSE) %f' %(np.sum((k_cv_train_true-reg_out_imag)**2)/k_cv_train_true.shape[0]))
        print('Testing with fold %d' %(i_fold))
        
        reg_out_imag = model_imag.forwardpropagate(k_cv_test)
        model_imag.backpropagation(k_cv_fest, reg_out_imag, k_cv_test_true)
        print('Test Error (SSE) %f' %(np.sum((k_cv_test-reg_out_imag)**2)/k_cv_test.shape[0]))
        

### Train Normally

In [ ]:
start_time = time.time()
for e in range(100):
    reg_out_real = model_real.forwardpropagate(k_train_real)
    model_real.backpropagation(k_train_real, reg_out_real, k_train_true_real)
    if e % 100 == 0:
        print(e)
print('MLP trains in %s seconds' %(time.time() - start_time))        

In [ ]:
start_time = time.time()
for e in range(100):
    reg_out_imag = model_imag.forwardpropagate(k_train_imag)
    model_imag.backpropagation(k_train_imag, reg_out_imag, k_train_true_imag)
    if e % 100 == 0:
        print(e)
print('MLP trains in %s seconds' %(time.time() - start_time))        